# Payday Loan Vendor Capture

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
g_key = os.environ.get('GKEY')

In [6]:
# Load the csv containing the zip codes into a dataframe
counties_zips_df = pd.read_csv("resources/CACountiesZips.csv", encoding="utf-8")
counties_zips_df.drop(columns='index', inplace=True)
counties_zips_df

,Population,Poverty Count,poverty_male,poverty_female,poverty_white_alone,poverty_black_alone,population_american_indian_alone,poverty_asian_alone,poverty_native_hawaiian_alone,poverty_two_or_more_races,...,poverty_family_single_female,employment_labor_force,employment_not_labor_force,employment_employed,employment_unemployed,Zipcode,Poverty Rate,City,County,Type
0,54760.0,17496.0,7700.0,9796.0,10023.0,1444.0,37.0,10.0,0.0,196.0,...,1665.0,23755.0,14832.0,20844.0,2911.0,90001,31.950329,Los Angeles,Los Angeles,Standard
1,49475.0,17116.0,7753.0,9363.0,8262.0,5242.0,76.0,22.0,2.0,216.0,...,1775.0,18883.0,15064.0,16323.0,2560.0,90002,34.595250,Los Angeles,Los Angeles,Standard
2,66200.0,25832.0,11625.0,14207.0,5648.0,7478.0,185.0,28.0,19.0,365.0,...,2737.0,26638.0,18813.0,22346.0,4263.0,90003,39.021148,Los Angeles,Los Angeles,Standard
3,62760.0,15350.0,7134.0,8216.0,5002.0,790.0,27.0,2494.0,10.0,148.0,...,1286.0,37615.0,14039.0,32207.0,5395.0,90004,24.458254,Los Angeles,Los Angeles,Standard
4,39548.0,10496.0,4904.0,5592.0,2296.0,392.0,77.0,2516.0,7.0,185.0,...,860.0,23323.0,8652.0,20520.0,2803.0,90005,26.539901,Los Angeles,Los Angeles,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,157.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,110.0,0.0,110.0,0.0,96136,0.000000,Wendel,Lassen,Standard
1547,3826.0,569.0,285.0,284.0,473.0,0.0,70.0,0.0,0.0,26.0,...,33.0,1602.0,1690.0,1270.0,325.0,96137,14.871929,Westwood,Lassen,Standard
1548,853.0,88.0,40.0,48.0,64.0,0.0,0.0,0.0,0.0,0.0,...,0.0,660.0,154.0,598.0,62.0,96146,10.316530,Olympic Valley,Placer,Standard
1549,28686.0,4443.0,2236.0,2207.0,3037.0,86.0,47.0,78.0,0.0,273.0,...,323.0,16574.0,7138.0,14377.0,2161.0,96150,15.488392,South Lake Tahoe,El Dorado,Standard


In [49]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [50]:
# Construct an empty dataframe to hold search results
payday_df = pd.DataFrame(columns=['Zip Code','Payday Vendor','Address','Lat','Lng'])

payday_df

,Zip Code,Payday Vendor,Address,Lat,Lng


In [51]:
# Create a variable to hold the base url for the search
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

#Create a variable to hold the index
indexes=0

# Create a for loop to iterrate through the zip codes
for index, row in counties_zips_df.iterrows():
    
    # Create a variable to hold the zip code
    zip = row['Zipcode']
    
    # Params dictionary to update each iteration
    params = {
        "query": f"payday+loan+in+{zip}",
        "radius": 0,
        "key": g_key
    }

    # Make request and store results
    payday_list = requests.get(base_url, params=params)

    # Convert to json
    payday_list = payday_list.json()

    # Create a for loop to write correct results to the dataframe filter out result not containing the searched zip code
    for x in payday_list["results"]:
        # Try-except to skip any that are missing data.
        try:
            test = x['formatted_address']
            if str(zip) in test:
                print("Payday loan vendor found... writing to dataframe")
                payday_df.loc[indexes, 'Zip Code'] = zip
                payday_df.loc[indexes, 'Payday Vendor'] = x['name']
                payday_df.loc[indexes, 'Address'] = x['formatted_address']
                payday_df.loc[indexes, 'Lat'] = x['geometry']['location']['lat']
                payday_df.loc[indexes, 'Lng'] = x['geometry']['location']['lng']
                indexes += 1
            else:
                print("Skipping location \ not in zipcode")
                pass

        except (KeyError, IndexError): 
            print("Missing field/result... skipping.")
        
payday_df.head()

Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode

Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping loca

Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in 

Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ 

Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping loca

...

Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skippi

Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Payday loan vendor found... writing to da

Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ 

Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in 

,Zip Code,Payday Vendor,Address,Lat,Lng
0,90001,ACE Cash Express,"1232 E Florence Ave, Los Angeles, CA 90001, Un...",33.9746,-118.253
1,90001,Easy Financial,"6003 Compton Ave, Los Angeles, CA 90001, Unite...",33.9856,-118.248
2,90002,ACE Cash Express,"10005 S Central Ave, Los Angeles, CA 90002, Un...",33.9455,-118.255
3,90002,The Check Cashing Place,"2284 E Firestone Blvd, Los Angeles, CA 90002, ...",33.9579,-118.231
4,90002,Continental Currency Services,"1669 E 103rd St, Los Angeles, CA 90002, United...",33.9439,-118.244


In [53]:
# Display the dataframe
payday_df

,Zip Code,Payday Vendor,Address,Lat,Lng
0,90001,ACE Cash Express,"1232 E Florence Ave, Los Angeles, CA 90001, Un...",33.9746,-118.253
1,90001,Easy Financial,"6003 Compton Ave, Los Angeles, CA 90001, Unite...",33.9856,-118.248
2,90002,ACE Cash Express,"10005 S Central Ave, Los Angeles, CA 90002, Un...",33.9455,-118.255
3,90002,The Check Cashing Place,"2284 E Firestone Blvd, Los Angeles, CA 90002, ...",33.9579,-118.231
4,90002,Continental Currency Services,"1669 E 103rd St, Los Angeles, CA 90002, United...",33.9439,-118.244
...,...,...,...,...,...
3159,96161,Plumas Bank,"11638 Donner Pass Rd, Truckee, CA 96161, Unite...",39.3265,-120.214
3160,96161,Sierra Mountain Mortgage Inc,"10098 Jibboom St # 201, Truckee, CA 96161, Uni...",39.3284,-120.186
3161,96161,Bank of the West,"11202 Donner Pass Rd, Truckee, CA 96161, Unite...",39.3264,-120.204
3162,96161,"Happy Investments, Inc. Truckee Ca","11260 Donner Pass Road C1 # 1014-G, Truckee, C...",39.3272,-120.206


In [54]:

payday_df.to_csv("resources/PaydayVendorLocations.csv", encoding="utf-8", index=False)

In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Payday Vendor}</dd>
<dt>Zip Code</dt><dd>{Zip Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
vendor_info = [info_box_template.format(**row) for index, row in payday_df.iterrows()]
locations = payday_df[["Lat", "Lng"]]

In [62]:
# Add marker layer
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=vendor_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))